In [70]:
import os
import sys

import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline

__dir__ = "/home/justpainm8/notebooks/ml/code_classif/"

In [71]:
from tqdm import tqdm
from subprocess import call

repos = {
    "https://github.com/Mrgemy95/Tensorflow-Project-Template.git",
    "https://github.com/ageitgey/face_recognition.git",
    "https://github.com/donnemartin/system-design-primer.git",
    "https://github.com/keras-team/keras.git",
    "https://github.com/toddmotto/public-apis.git",
    "https://github.com/Zulko/moviepy.git",
    "https://github.com/tensorflow/minigo.git",
    "https://github.com/facebookresearch/Detectron.git",
    "https://github.com/mherrmann/fbs-tutorial.git",
    "https://github.com/deepfakes/faceswap.git",
    "https://github.com/NullArray/AutoSploit.git",
    "https://github.com/mherrmann/fbs.git",
    "https://github.com/tensorflow/tensorflow.git",
    "https://github.com/scikit-learn/scikit-learn.git",
    "https://github.com/pascalabcnet/pascalabcnet.git",
    "https://github.com/bgbennyboy/USB-Disk-Ejector.git",
    "https://github.com/lkesteloot/turbopascal.git",
    "https://github.com/jdc0589/CaseConversion.git",
    "https://github.com/RomanYankovsky/DelphiAST.git",
    "https://github.com/salvadordf/CEF4Delphi.git",
    "https://github.com/BerndGabriel/HtmlViewer.git",
    "https://github.com/SynEdit/SynEdit.git",
    "https://github.com/FMXExpress/Firemonkey.git",
    "https://github.com/remobjects/pascalscript.git",
    "https://github.com/PascalCoin/PascalCoin.git",
    "https://github.com/alrieckert/lazarus.git",
    "https://github.com/danieleteti/delphistompclient.git",
    "https://github.com/Relfos/TERRA-Engine.git",
    "https://github.com/fastplaz/fastplaz.git"
}

for r in tqdm(repos):
    call(["git", "clone", r, os.path.join(__dir__, "code/{}".format(r.split('.')[-1]))])


100%|██████████| 29/29 [02:25<00:00,  5.01s/it]


In [74]:
import shutil
from threading import Thread

def get_language_files(path, ext={"cpp": ("cpp", "hpp"), "py": ("py",), "pas": ("pas",)}):
    counts = {ex: 0 for ex in ext.keys()}
    for root, dirs, files in tqdm(os.walk(path)):
        for f in files:
            exdir = f.split('.')[-1]
            if exdir not in ext:
                continue
            for e in ext[exdir]:
                if f.endswith(e):
                    counts[exdir] += 1
                    shutil.copy(os.path.join(root, f), os.path.join(__dir__, exdir))
                    break
    return counts

In [75]:
counts = get_language_files("/home/justpainm8/notebooks/ml/code_classif/code")
counts


0it [00:00, ?it/s]
8it [00:00, 65.44it/s]
49it [00:00, 220.42it/s]
293it [00:00, 905.87it/s]
780it [00:00, 1842.34it/s]
2382it [00:00, 4549.81it/s]
3005it [00:00, 4286.61it/s]
3547it [00:00, 3827.69it/s]
4159it [00:01, 4044.02it/s]
4656it [00:01, 4044.85it/s]
5125it [00:01, 3733.26it/s]
5517it [00:01, 3407.96it/s]
5855it [00:01, 3405.95it/s]
6182it [00:01, 3322.98it/s]
6481it [00:01, 3255.12it/s]
6759it [00:02, 3101.44it/s]
7059it [00:02, 3094.51it/s]
7339it [00:02, 3043.39it/s]
7606it [00:02, 3027.83it/s]
7856it [00:02, 2973.50it/s]
8144it [00:02, 2961.84it/s]
9824it [00:02, 3447.39it/s]
11033it [00:02, 3725.96it/s]
11831it [00:03, 3793.63it/s]
12542it [00:03, 3717.84it/s]
13280it [00:03, 3823.25it/s]
13908it [00:03, 3754.96it/s]
14430it [00:03, 3754.29it/s]
14907it [00:04, 3722.13it/s]
15330it [00:04, 3671.81it/s]
15701it [00:04, 3650.34it/s]
16173it [00:04, 3674.44it/s]
16607it [00:04, 3689.01it/s]
17163it [00:04, 3729.40it/s]
17686it [00:04, 3758.17it/s]
17703it [00:04, 3760.22it/

{'cpp': 15534, 'pas': 11663, 'py': 7279}

In [76]:
files = {ext: [] for ext in counts}
for fdir in counts:
    print(fdir)
    files[fdir] = os.listdir(fdir)
files

py
cpp
pas


{'cpp': ['t_5_007.cpp',
  'test_assert_fail_is_empty.cpp',
  'remove_varargs.cpp',
  'test_equal.cpp',
  't_2_004.cpp',
  'named_mutex_test.cpp',
  'log-formatter-test.cpp',
  'test_arithmetic_tommath.cpp',
  'perf_saxpy.cpp',
  'multifunction.cpp',
  'filt_attr.cpp',
  'perf_stl_set_union.cpp',
  'MsmComposite.cpp',
  'vector10_c.cpp',
  'test_has_template.cpp',
  'bfs-example.cpp',
  'test_assert_is_tuple.cpp',
  'mp_transform_q.cpp',
  'make_functions_test.cpp',
  'is_output_streamable_test.cpp',
  'comp_ellint_2.cpp',
  'books.cpp',
  'github_15.cpp',
  'lambda_tests1b2p.cpp',
  'test_erasure.cpp',
  'length_geo.cpp',
  'decl_exit_static_inv_none.cpp',
  'array_fail_spa_wpa_ma.cpp',
  'test_7868.cpp',
  'no_cxx17_inline_variables_pass.cpp',
  'test_shared_mutex_timed_locks.cpp',
  '11166-remove-race.cpp',
  'TimePicker.cpp',
  'no_cxx98_binders_pass.cpp',
  'assign_points.cpp',
  'sf_sin_pi_incl_test.cpp',
  'EumlInternal.cpp',
  'ImageRotationU.cpp',
  'test_typeof2.cpp',
  'uniqu

In [77]:
def make_data_list(files):
    text, labels = [], []
    for f in files:
        text.append(np.array(files[f]))
        labels.append(np.array([f] * len(files[f])))
    return np.hstack(text), np.hstack(labels)


def as_one_hot(x, classes=sorted(list(counts.keys()))):
    return [1 if x == c else 0 for c in classes]


def as_class_label(x, classes=sorted(list(counts.keys()))):
    return np.array(as_one_hot(x, classes)).argmax()


def from_label_to_class(label, classes=sorted(list(counts.keys()))):
    return classes[int(label)]



data = make_data_list(files)
df = pd.DataFrame(index=None, columns=['source', 'target'])
df['source'], df['target'] = data
df['target'] = df['target'].apply(lambda x: as_class_label(x))
df

,source,target
0,decorator_utils_test.py,2
1,label.py,2
2,decode_csv_op_test.py,2
3,profiler_test.py,2
4,datastructures.py,2
5,vector_laplace_diag_test.py,2
6,mocking.py,2
7,jupyter_notebook_config.py,2
8,views_broken.py,2
9,independent.py,2


In [78]:
class FileLoader(object):
    
    def __init__(self, n, path=__dir__):
        self.n = n
        self.path = path
        self.tq = tqdm(range(n))
        
    def load(self, filename):
        path = os.path.join(self.path, filename.split('.')[-1])
        with open(os.path.join(path, filename), 'r', encoding='utf-8', errors="ignore") as f:
            self.tq.update()
            return "\n".join(f.readlines())
loader = FileLoader(len(df['source'].values))
df['source'] = df['source'].apply(loader.load)


 99%|█████████▉| 23054/23331 [00:02<00:00, 7279.39it/s]

In [79]:
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


XGB_params = dict(
    n_estimators=128,
    n_jobs=8,
)

In [80]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    df['source'].values, 
    df['target'].values, 
    test_size=0.3
)

In [81]:
NB_params = {}

In [82]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


preprocess = [
    ('vect', CountVectorizer()), 
    ('tfidf', TfidfTransformer())
]

xgb_clf = Pipeline(preprocess + [('clf', xgb.XGBClassifier(**XGB_params))])
nvb_clf = Pipeline(preprocess + [('clf', MultinomialNB(**NB_params))])

In [83]:
xgb_clf.fit(X_train, y_train)
# nvb_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [102]:
class Ansamble(object):
    
    def __init__(self, clfs):
        self.clfs = clfs
        
    def predict_proba(self, data):
        return np.mean([clf.predict_proba(data) for clf in self.clfs], axis=0)
    
    def predict(self, data):
        return self.predict_proba(data).argmax(axis=1)
    
    def __call__(self, data):
        return self.predict(data)

In [84]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, xgb_clf.predict(X_test)))
print(accuracy_score(y_test, nvb_clf.predict(X_test)))

/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9992857142857143
0.9862857142857143


In [103]:
clf = Ansamble([xgb_clf, nvb_clf])
accuracy_score(y_test, clf.predict(X_test))

0.9987142857142857

In [104]:
from sklearn.externals import joblib
s = joblib.dump(xgb_clf, 'xgb_text_clf_cpp_py_pas_0.99.pkl')

100%|██████████| 14815/14815 [5:03:32<00:00,  1.23s/it]


In [106]:
py_code = '''import os
if __name__ == "__main__":
    print(*range(10), sep='faggot')'''
cpp_code = '''#include <iostream>
<class T>
T doubleIt(T t) {
    return t * 2;
}

void main() {
    std::cout << "Fuck you" << " and also doubleIt(10) = " << doubleIt(10);
    return 0;
}
'''
pas_code = """const N = 4;
var
  f1: text;
  j,i,k,l,endline : integer;
  
begin
assign(f1,'D:\in.txt');
reset(f1);
  while not Eoln(f1) do begin
    Readln(f1,endline);
    //Считываем 4 раза по 4 элемента
    inc(j);
  end;
  close(f1);
end"""
code_samples = [py_code, cpp_code, pas_code]
for code in code_samples:
    print('|{}|\n'.format('-' * 20))
    print('\033[92m\n{}\n\033[0m'.format(code))
    proba, pred = xgb_clf.predict_proba([code]), text_clf.predict([code])
    print(' ' * 3, proba.max(), from_label_to_class(pred))
    print('|{}|\n\n'.format('-' * 20))

|--------------------|


import os
if __name__ == "__main__":
    print(*range(10), sep='faggot')



/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


    0.9992268 py
|--------------------|


|--------------------|


#include <iostream>
<class T>
T doubleIt(T t) {
    return t * 2;
}

void main() {
    std::cout << "Fuck you" << " and also doubleIt(10) = " << doubleIt(10);
    return 0;
}




/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


    0.99910754 cpp
|--------------------|


|--------------------|


const N = 4;
var
  f1: text;
  j,i,k,l,endline : integer;
  
begin
assign(f1,'D:\in.txt');
reset(f1);
  while not Eoln(f1) do begin
    Readln(f1,endline);
    //Считываем 4 раза по 4 элемента
    inc(j);
  end;
  close(f1);
end

    0.99985003 pas
|--------------------|




/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [108]:
py_code = """def tryk(s):
    f


fin = open('grants.in', 'r')
fout = open('grants.out', 'w')
powers = [1]
for i in range(60):
    powers.append(powers[-1] * 2)
a = int(fin.readline())
for i in range(a):
    s = int(fin.readline())
    one = bin(s)[2:][::-1]
    maxx = '0'
    rem = one
    for u in range((s + 3) // 3):
        j = u * 3
        two = bin(s - j)[2:][::-1]
        three = bin(u)[2:][::-1]
        #print(one, two, three)
        if three[0] != two[0] or three[0] == '0':
            n = 1
            m = 1
            cont = True
            while n < len(two) and m < len(three):
                if two[n] == three[m] and two[n] == '1' and three[m - 1] == '1':
                    cont = False
                    #print(two, three, n, m)
                    break
                n += 1
                m += 1
            if cont:
                maxx = three
                rem = two
    num = int(maxx[::-1], 2)
    p1 = num
    p2 = num
    p3 = num
    ln = len(maxx)
    lr = len(rem)
    for j in range(lr):
        if rem[j] == '1':
            if j < ln and maxx[j] == '1':
                p2 += powers[j - 1]
                p3 += powers[j - 1]
            else:
                p1 += powers[j]
    print(p1, p2, p3, file = fout)
fin.close()
fout.close()"""
code_samples = [py_code]
for code in code_samples:
    print('|{}|\n'.format('-' * 20))
    print('\033[92m\n{}\n\033[0m'.format(code))
    proba, pred = xgb_clf.predict_proba([code]), text_clf.predict([code])
    print(' ' * 3, proba.max(), from_label_to_class(pred))
    print('|{}|\n\n'.format('-' * 20))

|--------------------|


def tryk(s):
    f


fin = open('grants.in', 'r')
fout = open('grants.out', 'w')
powers = [1]
for i in range(60):
    powers.append(powers[-1] * 2)
a = int(fin.readline())
for i in range(a):
    s = int(fin.readline())
    one = bin(s)[2:][::-1]
    maxx = '0'
    rem = one
    for u in range((s + 3) // 3):
        j = u * 3
        two = bin(s - j)[2:][::-1]
        three = bin(u)[2:][::-1]
        #print(one, two, three)
        if three[0] != two[0] or three[0] == '0':
            n = 1
            m = 1
            cont = True
            while n < len(two) and m < len(three):
                if two[n] == three[m] and two[n] == '1' and three[m - 1] == '1':
                    cont = False
                    #print(two, three, n, m)
                    break
                n += 1
                m += 1
            if cont:
                maxx = three
                rem = two
    num = int(maxx[::-1], 2)
    p1 = num
    p2 = num
    p3 = num
    ln = len

/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [25]:
class Stream(object):

    def __init__(self, filename=None):
        self.filename = filename

    def write(self, *args, **kwargs):
        pass

    def writeln(self, *args):
        pass


class STDOUTStream(Stream):

    def write(self, *args):
        print(*args)

    def writeln(self, *args):
        return self.write(*args)


class FileStream(Stream):

    def __init__(self, filename):
        super().__init__(filename)

        self.file = open(filename, 'w')

    def write(self, s, *args):
        self.file.write(s)

    def writeln(self, s, *args):
        self.file.write(s + "\n")


    def __enter__(self):
        self.file.__enter__()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.file.__exit__(exc_type, exc_val, exc_tb)

    def __del__(self):
        if not self.file.closed:
            self.file.close()

In [31]:
sorted(list(counts.keys()))

['cpp', 'pas', 'py']

In [123]:
class Solver(object):

    def __init__(self, pipeline, classes):
        self.clf = pipeline
        self.classes = classes

    def as_one_hot(self, x):
        return [1 if x == c else 0 for c in self.classes]

    def as_class_label(self, x):
        return np.array(self.as_one_hot(x)).argmax()

    def from_label_to_class(self, label):
        return self.classes[int(label)]

    def classify(self, code):
        if isinstance(code, str):
            return self.from_label_to_class(self.clf.predict(np.array([code])))
        return [self.from_label_to_class(l) for l in self.clf.predict(code)]

    def __call__(self, code):
        return self.classify(code)


filename = "E2"
stream = STDOUTStream() # FileStream("ans.txt")

path = "School-Stuff/MOSh/{}".format(filename)
solver = Solver(xgb_clf, classes=("C++", "Pascal", "Python"))
with FileStream("School-Stuff/MOSh/answers/{}.txt".format(filename)) as stream2:
    for file in tqdm(sorted(int(e) for e in os.listdir(path))):
        with open(os.path.abspath(os.path.join(path, "{}".format(file))), 'r', errors='ignore') as f:
            # print("\n".join(f.readlines()))
            # print(solver('\n'.join(f.readlines())))
            # break
            stream2.writeln(solver('\n'.join(f.readlines())))



  0%|          | 0/700 [00:00<?, ?it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


  0%|          | 1/700 [00:00<03:19,  3.51it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


  0%|          | 2/700 [00:00<03:07,  3.73it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an arra

  3%|▎         | 24/700 [00:05<02:35,  4.36it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


  4%|▎         | 25/700 [00:05<02:34,  4.36it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


  4%|▎         | 26/700 [00:05<02:35,  4.33it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check tha

  7%|▋         | 48/700 [00:11<02:29,  4.36it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


  7%|▋         | 49/700 [00:11<02:29,  4.36it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


  7%|▋         | 50/700 [00:11<02:29,  4.36it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check tha

 10%|█         | 72/700 [00:16<02:24,  4.35it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 10%|█         | 73/700 [00:16<02:24,  4.35it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 11%|█         | 74/700 [00:16<02:23,  4.36it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check tha

 14%|█▎        | 96/700 [00:21<02:18,  4.37it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 14%|█▍        | 97/700 [00:22<02:17,  4.37it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 14%|█▍        | 98/700 [00:22<02:17,  4.37it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check tha

...

 86%|████████▌ | 600/700 [02:16<00:22,  4.39it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 86%|████████▌ | 601/700 [02:16<00:22,  4.39it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 86%|████████▌ | 602/700 [02:16<00:22,  4.39it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check 

 89%|████████▉ | 624/700 [02:22<00:17,  4.39it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 89%|████████▉ | 625/700 [02:22<00:17,  4.39it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 89%|████████▉ | 626/700 [02:22<00:16,  4.39it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check 

 93%|█████████▎| 648/700 [02:27<00:11,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 93%|█████████▎| 649/700 [02:27<00:11,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 93%|█████████▎| 650/700 [02:27<00:11,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check 

 96%|█████████▌| 672/700 [02:32<00:06,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 96%|█████████▌| 673/700 [02:33<00:06,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 96%|█████████▋| 674/700 [02:33<00:05,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check 

 99%|█████████▉| 696/700 [02:38<00:00,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


100%|█████████▉| 697/700 [02:38<00:00,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


100%|█████████▉| 698/700 [02:38<00:00,  4.40it/s]/home/justpainm8/.conda/envs/cenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check 